In [5]:
from csv import reader                          # get the csv reader
infile = open("SFPD_Incidents_-_from_1_January_2003.csv", 'r')    # open the file for reading
fileValues = []

first = True 
for line in reader( infile ):
    if not first:
        incident_number, category, description, day_of_week, date, time, district, resulution, address, x, y , location, id = line
        
        values = {'incident_number': incident_number, 'category': category, 'description': description, 'day_of_week': day_of_week, 
                  'date': date, 'time': time, 'district': district, 'resulution': resulution, 'address': address, 'x': x, 'y': y , 
                  'location': location, 'id': id}
        if float(y) != 90.0:
            fileValues.append(values)
    first = False

In [6]:
prostitutions = []

for line in fileValues:
    if line['district'] is not '' and line['category'] == 'PROSTITUTION':
        line['location'] = float(line['y']), float(line['x'])
        prostitutions.append(line)

In [7]:
from sklearn.cluster import KMeans

estimators = {}

for k in range(2,11):
    cls = KMeans(n_clusters=k)
    cls.fit([line['location'] for line in prostitutions])
    estimators[k] = cls

In [8]:
import csv
from collections import OrderedDict

k_values=OrderedDict()
for k in range(2,7):
    k_values[k] = estimators[k].predict([line['location'] for line in prostitutions])

chosenFileValues = []
i = 0
for crime in prostitutions:
    values = OrderedDict()
    values = {'lon': float(crime['x']), 'lat': float(crime['y']), 'k2': k_values[2][i], 'k3': k_values[3][i], 
              'k4': k_values[4][i], 'k5': k_values[5][i], 'k6': k_values[6][i]}
    chosenFileValues.append(values)
    i = i+1

keys = chosenFileValues[0].keys()
with open('week8_part2.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(chosenFileValues)